# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('tic-tac-toe.csv')
print(df.head())

  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True


In [2]:
# Check basic information about the dataset
print(df.info())

# Display unique values in each column
for column in df.columns:
    print(f"{column} unique values: {df[column].unique()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None
TL unique values: ['x' 'o' 'b']
TM unique values: ['x' 'o' 'b']
TR unique values: ['x' 'o' 'b']
ML unique values: ['x' 'o' 'b']
MM unique values: ['o' 'b' 'x']
MR unique values: ['o' 'b' 'x']
BL unique values: ['x' 'o' 'b']
BM unique values: ['o' 'x' 'b']
BR unique values: ['o' 'x' 'b']
class unique values: [ True False]


In [3]:
# Map categorical values to numeric
mapping = {'x': 1, 'o': 0, 'b': -1, True: 1, False: 0}
df = df.map(lambda val: mapping[val] if val in mapping else val)

print(df.head())

   TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0   1   1   1   1   0   0   1   0   0      1
1   1   1   1   1   0   0   0   1   0      1
2   1   1   1   1   0   0   0   0   1      1
3   1   1   1   1   0   0   0  -1  -1      1
4   1   1   1   1   0   0  -1   0  -1      1


In [4]:
# Separate input features (X) and the output label (y)
X = df.iloc[:, :-1]  # All columns except the last one
y = df.iloc[:, -1]   # The last column

In [5]:
from sklearn.preprocessing import MinMaxScaler

# Normalize inputs
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

print(X_normalized[:5])  # Display the first 5 rows of normalized data

[[1.  1.  1.  1.  0.5 0.5 1.  0.5 0.5]
 [1.  1.  1.  1.  0.5 0.5 0.5 1.  0.5]
 [1.  1.  1.  1.  0.5 0.5 0.5 0.5 1. ]
 [1.  1.  1.  1.  0.5 0.5 0.5 0.  0. ]
 [1.  1.  1.  1.  0.5 0.5 0.  0.5 0. ]]


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [6]:
# Import train_test_split for splitting data
from sklearn.model_selection import train_test_split  

# Split the data into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Print the shapes of training and test sets
print(f"Training data shape: {X_train.shape}, Test data shape: {X_test.shape}")

Training data shape: (766, 9), Test data shape: (192, 9)


In [7]:
 # Import Sequential model
from tensorflow.keras.models import Sequential 

# Initialize the Sequential model
model = Sequential()

In [8]:
# Import Input for defining the input layer
from tensorflow.keras.layers import Input, Dense  

# Add an Input layer to define the input shape
model.add(Input(shape=(9,)))  # Input layer for 9 grid features

# Add a first hidden layer with 64 neurons and ReLU activation
model.add(Dense(64, activation='relu'))  

# Add a second hidden layer with 32 neurons and ReLU activation
model.add(Dense(32, activation='relu'))  

# Add the output layer with 2 neurons (for True/False classes) and Softmax activation
model.add(Dense(2, activation='softmax'))

In [9]:
# Compile the model with optimizer, loss function, and metrics
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model on the training data
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4878 - loss: 0.7059 - val_accuracy: 0.6818 - val_loss: 0.6307
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6303 - loss: 0.6525 - val_accuracy: 0.6753 - val_loss: 0.6296
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6651 - loss: 0.6252 - val_accuracy: 0.6753 - val_loss: 0.6194
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6389 - loss: 0.6362 - val_accuracy: 0.6818 - val_loss: 0.6120
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6566 - loss: 0.6144 - val_accuracy: 0.6883 - val_loss: 0.6107
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6854 - loss: 0.6009 - val_accuracy: 0.7078 - val_loss: 0.6066
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6824 - loss: 0.5942 - val_accuracy: 0.7143 - val_loss: 0.6027
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6834 - loss: 0.5933 - val_accuracy: 0.7143 - val_loss:

In [11]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8358 - loss: 0.3891 
Test Loss: 0.39379942417144775, Test Accuracy: 0.8385416865348816


In [15]:
# Save the trained model to a file
model.save('tic-tac-toe.keras')
print("Model saved as tic-tac-toe.keras")

Model saved as tic-tac-toe.keras


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

	1.	Load the saved model.
	2.	Use the model to make predictions on a few random rows from the test dataset.
	3.	Compare the predicted labels with the actual labels to check if the predictions are correct.

In [16]:
from tensorflow.keras.models import load_model  # Import load_model to load the saved model
import numpy as np  # Import numpy for handling array operations

# Step 1: Load the saved model
model = load_model('tic-tac-toe.keras')  # Load the previously saved model
print("Model loaded successfully.")

# Step 2: Select a few random rows from the test dataset
random_indices = np.random.choice(X_test.shape[0], size=5, replace=False)  # Pick 5 random rows
X_sample = X_test[random_indices]  # Extract the corresponding feature rows
y_sample = y_test.iloc[random_indices]  # Extract the corresponding true labels

# Step 3: Make predictions
predictions = model.predict(X_sample)  # Predict the class probabilities
predicted_labels = np.argmax(predictions, axis=1)  # Convert probabilities to class labels (0 or 1)

# Step 4: Compare predictions with actual labels
print("Random Samples from Test Set:")
for i in range(len(X_sample)):
    print(f"Input: {X_sample[i]} | Predicted: {predicted_labels[i]} | Actual: {y_sample.iloc[i]}")

Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Random Samples from Test Set:
Input: [1.  0.5 1.  1.  0.  0.  1.  0.5 0.5] | Predicted: 1 | Actual: 1
Input: [1.  0.5 0.5 0.  0.5 1.  1.  0.5 1. ] | Predicted: 0 | Actual: 0
Input: [1.  0.5 1.  1.  0.5 0.5 0.5 1.  1. ] | Predicted: 0 | Actual: 0
Input: [0.5 0.  1.  1.  0.5 0.  1.  0.  0.5] | Predicted: 1 | Actual: 0
Input: [1.  0.5 0.  0.5 0.  0.5 1.  1.  1. ] | Predicted: 1 | Actual: 1


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam  # Import Adam optimizer for custom learning rate

# Step 1: Create a new model with more layers
model = Sequential()

# Input layer
model.add(Input(shape=(9,)))  # Input layer for 9 grid features

# Hidden layers
model.add(Dense(128, activation='relu'))  # First hidden layer with 128 neurons
model.add(Dense(64, activation='relu'))  # Second hidden layer with 64 neurons
model.add(Dense(32, activation='relu'))  # Third hidden layer with 32 neurons

# Output layer
model.add(Dense(2, activation='softmax'))  # Output layer with 2 classes (True/False)

In [18]:
# Step 2: Adjust the learning rate
learning_rate = 0.001  # Specify a custom learning rate
optimizer = Adam(learning_rate=learning_rate)  # Create an Adam optimizer with the custom learning rate

# Compile the model with the custom optimizer
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [19]:
# Step 3: Train the model with more epochs
epochs = 100  # Increase the number of epochs for training
history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_split=0.2)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6257 - loss: 0.6647 - val_accuracy: 0.6818 - val_loss: 0.6164
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6429 - loss: 0.6329 - val_accuracy: 0.6818 - val_loss: 0.6152
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6656 - loss: 0.6090 - val_accuracy: 0.7078 - val_loss: 0.6046
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6593 - loss: 0.6019 - val_accuracy: 0.7143 - val_loss: 0.5998
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6823 - loss: 0.5805 - val_accuracy: 0.6818 - val_loss: 0.5934
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7133 - loss: 0.5530 - val_accuracy: 0.7078 - val_loss: 0.5829
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7535 - loss: 0.5343 - val_accuracy: 0.7208 - val_loss: 0.5679
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7644 - loss: 0.5079 - val_accuracy: 0.7403 - v

In [20]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9124 - loss: 0.2914 
Test Loss: 0.36504778265953064, Test Accuracy: 0.9010416865348816


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here